# Analysis of Ficus RAD-seq data

### Table of contents
[Software installation (conda)](#Required-software)  
[The assembled RAD data](#The-assembled-data-sets)  
[Phylogenetic analysis (raxml)](#Analysis-BPP)  
[Plot results](#Plots)



## Required software
All required software can be installed locally using *conda*. I assume here that you already have `ipyrad` installed using conda. 

In [1]:
## conda install toytree   -c eaton-lab
## conda install ipyrad    -c ipyrad 
## conda install bpp       -c ipyrad 

In [2]:
## import packages
import ipyrad as ip
import ipyrad.analysis as ipa
import numpy as np
import toyplot
import toytree
import glob

## print ipyrad info
print "ipyrad v.{}".format(ip.__version__)

ipyrad v.0.6.20


## Cluster setup
see more information on ipyparallel setup here. 

In [3]:
## print ipyparallel cluster information
import ipyparallel as ipp
ipyclient = ipp.Client()
print ip.cluster_info(ipyclient)

host compute node: [40 cores] on tinus


### Assembled data sets

In [5]:
## create subsampled pharma-clade branch
pharma = ip.load_json("analysis-ipyrad/pharma_dhi_s4.json")
america = ip.load_json("analysis-ipyrad/america_dhi_s4.json")

  loading Assembly: pharma_dhi_s4
  from saved path: ~/Documents/Ficus/analysis-ipyrad/pharma_dhi_s4.json
  loading Assembly: america_dhi_s4
  from saved path: ~/Documents/Ficus/analysis-ipyrad/america_dhi_s4.json


# Analysis BPP

#### Pharmacosyceae clade

In [6]:
## a tree hypothesis (guidetree) (here based on tetrad results)
newick = "((((glabrata, insipida), yoponensis), maxima), tonduzii);"

## a dictionary mapping sample names to 'species' names
imap = {
    "glabrata": ["B133_glabrata", "A97_glabrata", "B134_glabrata", "B131_glabrataXmaxima"],
    "insipida": ["A95_insipida", "B127_insipida", "C15_insipida", 
                 "B128_insipida", "B127_insipida", "A95_insipida"],
    "yoponensis": ["C45_yoponensis", "C47_yoponensis", "C46_yoponensis"],
    "maxima": ["A94_maxima", "C17_maxima", "B119_maxima", "B120_maxima"],
    "tonduzii": ["C48_tonduzii"],
    }

## loci must have data for at least N samples in each species.
minmap = {
    "glabrata": 4,
    "insipida": 4,
    "yoponensis": 3,
    "maxima": 4, 
    "tonduzii": 1,
    }

In [7]:
## check your (starting) tree hypothesis
toytree.tree(newick).draw();

tonduzii maxima yoponensis glabrata insipida 
 
 
 Save as .csv

In [12]:
## create a bpp object to run algorithm 00
boo = ipa.bpp(
    locifile=pharma.outfiles.loci,
    guidetree=newick, 
    imap=imap, 
    minmap=minmap,   
    workdir="analysis-bpp/",
    )

In [13]:
## set some optional params, leaving others at their defaults
boo.params.burnin = 10000
boo.params.nsample = 50000
boo.params.sampfreq = 25
boo.params


burnin          10000               
cleandata       0                   
copied          False               
delimit_alg     (0, 5)              
finetune        (0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01)
infer_delimit   0                   
infer_sptree    0                   
nsample         50000               
sampfreq        25                  
seed            12345               
tauprior        (2, 2000, 1)        
thetaprior      (2, 2000)           
usedata         1                   

In [14]:
## set some optional filters leaving others at their defaults
boo.filters.maxloci=500
boo.filters.minsnps=4

## print filters
boo.filters

maxloci   500                 
minmap    {'insipida': 4, 'tonduzii': 1, 'maxima': 4, 'glabrata': 4, 'yoponensis': 3}
minsnps   4                   

In [15]:
## write files 
boo.write_bpp_files(prefix="pharma-ltest")

input files created for job pharma-ltest (500 loci)


In [17]:
boo.submit_bpp_jobs(
    prefix="pharma-ltest", 
    nreps=3, 
    ipyclient=ipyclient, 
    seed=12345, 
    randomize_order=True,
    )

submitted 3 bpp jobs [pharma-ltest] (500 loci)


## Plots